# MIHCSME Metadata Upload - Simple Workflow

This notebook demonstrates a straightforward approach to uploading MIHCSME metadata to OMERO.

## Steps:
1. Connect to OMERO
2. Load and validate MIHCSME Excel file
3. Upload metadata to Screen and Wells
4. Verify the upload

In [ ]:
# Import required libraries
from omero.gateway import BlitzGateway
import os
from dotenv import load_dotenv
import logging

# Import our MIHCSME uploader
import src.omero_metadata_uploader as omu
import importlib
importlib.reload(omu)

# Set up logging for better feedback
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

print("✅ Libraries imported successfully")

In [ ]:
# 1. Connect to OMERO
load_dotenv(override=True)

conn = BlitzGateway(
    host=os.environ.get("HOST"),
    username=os.environ.get("USER_NAME"),
    passwd=os.environ.get("PASSWORD"),
    port=os.environ.get("PORT"),
    secure=True,
)

connected = conn.connect()
if connected:
    conn.c.enableKeepAlive(60)
    print(f"✅ Connected to OMERO as {conn.getUser().getName()}")
else:
    print("❌ Failed to connect to OMERO")

In [ ]:
# 2. Configure your upload
EXCEL_FILE = "MIHCSME Template.xlsx"  # Path to your MIHCSME Excel file
SCREEN_ID = 151                        # Your target Screen ID
NAMESPACE = "metadata"                # Namespace for annotations

print(f"📋 Configuration:")
print(f"   Excel file: {EXCEL_FILE}")
print(f"   Target Screen ID: {SCREEN_ID}")
print(f"   Namespace: {NAMESPACE}")

In [ ]:
# 3. Load and validate the MIHCSME file
print("📖 Loading MIHCSME Excel file...")

# Basic file validation
validation_result = omu.validate_mihcsme_file_structure(EXCEL_FILE)
if validation_result['is_valid']:
    print("✅ MIHCSME file structure is valid")
else:
    print("❌ MIHCSME file has errors:")
    for error in validation_result['errors']:
        print(f"   - {error}")

# Convert Excel to JSON format
metadata_json = omu.convert_excel_to_json(EXCEL_FILE)
print(f"✅ Loaded metadata with {len(metadata_json.get('AssayConditions', []))} well conditions")

In [ ]:
# 4. Upload metadata to OMERO
print("🚀 Uploading metadata to OMERO...")

result = omu.annotate_omero_object(
    conn, 
    target_object_type="Screen", 
    target_object_id=SCREEN_ID, 
    metadata_json=metadata_json, 
    base_namespace=NAMESPACE,
    replace=True
)

print(f"\n📊 Upload Results:")
print(f"   Status: {result['status']}")
print(f"   Message: {result['message']}")
print(f"   Wells processed: {result['wells_processed']}")
print(f"   Wells succeeded: {result['wells_succeeded']}")
print(f"   Wells failed: {result['wells_failed']}")

if result['status'] == 'success':
    print("✅ Metadata upload completed successfully!")
else:
    print("⚠️ Upload completed with some issues. Check the logs above.")

In [ ]:
# 6. Check well-level metadata
print("🧪 Checking well-level metadata...")

# Get the first plate in the screen
plates = list(screen.listChildren())
if plates:
    plate = plates[0]
    print(f"Checking plate: {plate.getName()}")
    
    # Get first few wells
    wells = list(plate.listChildren())[:5]
    wells_with_metadata = 0
    
    for well in wells:
        well_annotations = [ann for ann in well.listAnnotations() 
                          if hasattr(ann, 'getNs') and ann.getNs() and ann.getNs().startswith(NAMESPACE)]
        
        if well_annotations:
            wells_with_metadata += 1
            row, col = well.row, well.column
            well_name = f"{chr(ord('A') + row)}{col + 1:02d}"
            print(f"   Well {well_name}: {len(well_annotations)} metadata annotations")
    
    print(f"\n✅ {wells_with_metadata} out of {len(wells)} wells checked have metadata")
else:
    print("❌ No plates found in the screen")

## Close connection


In [ ]:
# Close OMERO connection
conn.close()
print("👋 OMERO connection closed")